In [1]:
import warnings

import numpy as np
import pandas as pd

import statsmodels.api as sm
from statsmodels.stats.diagnostic import het_breuschpagan
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import QuantileTransformer

import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [4]:
df = pd.read_csv('https://raw.githubusercontent.com/Gaukhar-ai/for_my_Thinkful_work/master/capstone/NY_property_sale/nyc-rolling-sales.csv')
df.head()

,Unnamed: 0,BOROUGH,NEIGHBORHOOD,BUILDING CLASS CATEGORY,TAX CLASS AT PRESENT,BLOCK,LOT,EASE-MENT,BUILDING CLASS AT PRESENT,ADDRESS,...,RESIDENTIAL UNITS,COMMERCIAL UNITS,TOTAL UNITS,LAND SQUARE FEET,GROSS SQUARE FEET,YEAR BUILT,TAX CLASS AT TIME OF SALE,BUILDING CLASS AT TIME OF SALE,SALE PRICE,SALE DATE
0,4,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,392,6,,C2,153 AVENUE B,...,5,0,5,1633,6440,1900,2,C2,6625000,2017-07-19 00:00:00
1,5,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,26,,C7,234 EAST 4TH STREET,...,28,3,31,4616,18690,1900,2,C7,-,2016-12-14 00:00:00
2,6,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2,399,39,,C7,197 EAST 3RD STREET,...,16,1,17,2212,7803,1900,2,C7,-,2016-12-09 00:00:00
3,7,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2B,402,21,,C4,154 EAST 7TH STREET,...,10,0,10,2272,6794,1913,2,C4,3936272,2016-09-23 00:00:00
4,8,1,ALPHABET CITY,07 RENTALS - WALKUP APARTMENTS,2A,404,55,,C2,301 EAST 10TH STREET,...,6,0,6,2369,4615,1900,2,C2,8000000,2016-11-17 00:00:00


tax class for present to drop
how many neighborhoods do i have, convert to latitude and len
remove unnamed
add a new column NYC to the address city with the NYC. ----

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84548 entries, 0 to 84547
Data columns (total 22 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Unnamed: 0                      84548 non-null  int64 
 1   BOROUGH                         84548 non-null  int64 
 2   NEIGHBORHOOD                    84548 non-null  object
 3   BUILDING CLASS CATEGORY         84548 non-null  object
 4   TAX CLASS AT PRESENT            84548 non-null  object
 5   BLOCK                           84548 non-null  int64 
 6   LOT                             84548 non-null  int64 
 7   EASE-MENT                       84548 non-null  object
 8   BUILDING CLASS AT PRESENT       84548 non-null  object
 9   ADDRESS                         84548 non-null  object
 10  APARTMENT NUMBER                84548 non-null  object
 11  ZIP CODE                        84548 non-null  int64 
 12  RESIDENTIAL UNITS               84548 non-null

In [8]:
df.isnull().sum()

Unnamed: 0                        0
BOROUGH                           0
NEIGHBORHOOD                      0
BUILDING CLASS CATEGORY           0
TAX CLASS AT PRESENT              0
BLOCK                             0
LOT                               0
EASE-MENT                         0
BUILDING CLASS AT PRESENT         0
ADDRESS                           0
APARTMENT NUMBER                  0
ZIP CODE                          0
RESIDENTIAL UNITS                 0
COMMERCIAL UNITS                  0
TOTAL UNITS                       0
LAND SQUARE FEET                  0
GROSS SQUARE FEET                 0
YEAR BUILT                        0
TAX CLASS AT TIME OF SALE         0
BUILDING CLASS AT TIME OF SALE    0
SALE PRICE                        0
SALE DATE                         0
dtype: int64

plan:
    transform '-' to nulls
    check if they are the same info: BUILDING CLASS AT PRESENT vs BUILDING CLASS AT TIME OF SALE drop at time of sale
    drop one of them
    check if address and block has same effect corr on the price, drop address is too specific convert the neighborhood to geographical coordinates: 
    if yes, drop one of them
    drop unnamed, just leave numbers 
    BOROUGH	NEIGHBORHOOD check if this is the same info, if yes, drop neighborhood
    see how many missing values in sale price and what to do with them if they're a lot
    made building class category to numeric
  make 15, 16 into int pd.to_numeric[ ] check what characters i have in them before converting them into Nas

In [ ]:
borough one hot encode. 
check the category for building class category
check what the building class
time class at present doesn't affect the past sale price
present and sale price today are same
make years into quarters, months aligns with the seasons. 
drop easement


do linear regression.
is it performing well - is it predicting prices right?
make a train_test_split with no feeding Y - answer
Here you see how it did on the test = past history of prices, how did it predict, 
then see how did it do on the actual prediction.
deploy the model, see how it works in the market. 
if it was expensive or not "expensive" then Logistic

In [1]:
df['SALE PRICE'].

NameError: name 'df' is not defined

In [2]:
!pip install geopy


You should consider upgrading via the 'c:\users\gaukharjavarova\appdata\local\programs\python\python38-32\python.exe -m pip install --upgrade pip' command.


In [12]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(user_agent="Humboldt")
location = geolocator.geocode("1524 North Kedzie ave Chicago")
print(location.address)
#Flatiron Building, 175, 5th Avenue, Flatiron, New York, NYC, New York, ...
print((location.latitude, location.longitude))
#(40.7410861, -73.9896297241625)
print(location.raw)
#{'place_id': '9167009604', 'type': 'attraction', ...}

1524, North Kedzie Avenue, Humboldt Park, Chicago, Cook County, Illinois, 60647, United States of America
(41.90904465, -87.70743364534412)
{'place_id': 125033599, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 163882200, 'boundingbox': ['41.9090217', '41.9091002', '-87.7076565', '-87.7071975'], 'lat': '41.90904465', 'lon': '-87.70743364534412', 'display_name': '1524, North Kedzie Avenue, Humboldt Park, Chicago, Cook County, Illinois, 60647, United States of America', 'class': 'building', 'type': 'yes', 'importance': 0.5209999999999999}


In [ ]:
lat, long

df

df_model

df_present

df['Address'].apply(gecode)